In [1]:
from sklearn.metrics import classification_report
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.utils.layer_utils import print_summary
from tensorflow.python.keras.models import Model as KerasModel
from tensorflow.python.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Reshape, Concatenate, LeakyReLU
from tensorflow.python.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
import os
import random
random.seed(32)
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" # Choose GPU NUMBERS [0, 1, 2, 3]
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)
sess.run(tf.global_variables_initializer())

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        # shear_range=0.2,
        # zoom_range=0.2,
        # horizontal_flip=True
)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
)
train_generator = train_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/train',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=True,
        seed=32) 
validation_generator = validation_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/val',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=True,
        seed=32) 

In [3]:
x=train_generator.__getitem__(0)
print(len(train_generator.labels),train_generator.labels)
print(x[1].shape)
print(x[1])
plt.imshow(x[0][7])
plt.title(x[1][7])


In [4]:
def get_pred(predictions):
    pred=[]
    for p in predictions:
        if p<0.50:
             pred.append(0)
        else:
            pred.append(1)
    return pred

In [5]:
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model_checkpoint_callback = ModelCheckpoint(
    filepath="XCE_DFD_Ebest.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
csv_logger = CSVLogger("XCE_DFD_train.csv", append=True, separator=',')
print_summary(model, line_length=150, positions=None, print_fn=None)

In [6]:
model.fit_generator(train_generator,
                    verbose=1,
                    epochs=50,
                    validation_data=validation_generator,
                    callbacks=[model_checkpoint_callback,csv_logger])
model.save_weights("XCE_DFD_E50.h5")

# DeepFakeDetection Test 

In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakeDetection/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFake Test 

In [8]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFake/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# FaceSwap Test 

In [9]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/FaceSwap/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# Face2Face Test 

In [10]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/Face2Face/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# NeuralTextures Test 

In [11]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/NeuralTextures/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DeepFakewild Test 

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DeepFakewild/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)

# DFDC Test 

In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)
input_tensor = Input(shape=(240, 240, 3))
model=tf.keras.applications.Xception(input_tensor=input_tensor,
    include_top=True, weights=None, classes=2)
model.load_weights("XCE_DFD_Ebest.h5")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=0.00005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
print_summary(model, line_length=150, positions=None, print_fn=None)
test_generator = test_datagen.flow_from_directory(
        'datasets/DFDC/test',# this is the target directory
        target_size=(240, 240),  # all images will be resized to 150x150
        class_mode='categorical',
        shuffle=False,
        seed=32) 
predictions=model.evaluate_generator(test_generator,verbose=2)
print(predictions)
predictions=model.predict_generator(test_generator,verbose=2)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes
report = classification_report(true_classes, predicted_classes, digits=4)
print(report)